In [1]:
#Libraries
import pandas as pd
import numpy as np
import random
from sklearn.metrics import pairwise
import matplotlib.pyplot as plt

In [2]:
#Dataset Read
dataset_df = pd.read_csv('data/jm1.csv')

In [3]:
#Mutator Function:
def seen(dataset,parent_candidate):
    size = dataset.shape[0]
    candidate_1 = dataset.loc[random.randint(0,size-1)]
    while candidate_1.equals(parent_candidate):
        candidate_1 = dataset.loc[random.randint(0,size-1)]
    candidate_2 = dataset.loc[random.randint(0,size-1)]
    while candidate_2.equals(parent_candidate) or candidate_1.equals(candidate_2):
        candidate_2 = dataset.loc[random.randint(0,size-1)]
    return parent_candidate,candidate_1,candidate_2
    
def select_candiate(dataset):
    size = dataset.shape[0]
    parent_candidate = dataset.loc[random.randint(0,size-1)]
    return seen(dataset,parent_candidate)

def data_mutator(dataset, f,cr,size):
    new_candidate = pd.DataFrame()
    dataset_columns = dataset.columns.values
    response_attribute = dataset_columns[len(dataset_columns)-1]
    for i in range(size):
        parent_candidate, child_candidate_1, child_candidate_2 = select_candiate(dataset)
        for key,value in parent_candidate.items():
            if key != response_attribute:
                if isinstance(parent_candidate[key], bool):
                    new_candidate.loc[i,key] = parent_candidate[key] if cr < random.random() else not parent_candidate[key]
                elif isinstance(parent_candidate[key], str):
                    new_candidate.loc[i,key] = random.choice([parent_candidate[key],child_candidate_1[key],child_candidate_2[key]])
                elif isinstance(parent_candidate[key], list):
                    temp_lst = []
                    for i, each in enumerate(parent_candidate[key]):
                          temp_lst.append(parent_candidate[key][i] if cr < random.random() else
                                          int(parent_candidate[key][i] +
                                              f * (child_candidate_1[key][i] - child_candidate_2[key][i])))
                    new_candidate.loc[i,key] = temp_lst
                else:
                    new_candidate.loc[i,key] = parent_candidate[key] if cr < random.random() else (
                      parent_candidate[key] + f * (child_candidate_1[key] - child_candidate_2[key]))
            else:
                new_candidate.loc[i,key] = parent_candidate[key]
    return new_candidate

In [4]:
new_data = data_mutator(dataset_df,0.3,0.75,100)
print(new_data)

       loc   v(g)  ev(g)  iv(g)       n          v      l        d        i  \
0    107.9   27.3   14.0   10.8   -52.2   -271.932 -0.009    0.000   -7.998   
1      1.3    1.0    1.0    1.3    17.0     58.810  0.443    2.290   24.218   
2      3.4    0.7    1.0    0.7     7.0    -32.235  0.542    2.000    4.973   
3     21.7    5.0    1.0    1.3    56.1    240.229  0.080   14.925   11.720   
4      5.0    1.0    1.0    1.0    14.7     51.890  0.391    2.920   17.649   
5     20.0    3.0    1.0    2.3    41.4    140.180  0.064    9.465   19.425   
6     15.8    5.0    2.2    3.5    16.0     45.840  0.120    3.450   13.180   
7      8.4    5.4    3.0    4.7    10.2     58.600  0.220    3.300    8.331   
8      2.5    0.4    1.0    0.7    14.0    -18.614  0.280    0.786   10.198   
9      9.4    1.0    1.0    1.0     0.0    132.354 -0.090    0.870   25.017   
10    -2.8    1.6    1.6    1.0   -43.1   -209.719  0.305  -15.677   20.291   
11    23.0    1.7    1.0    1.7    44.6    199.585  

In [5]:
count = 0
for i in new_data.defects:
    if i:
        count+=1
print(count)

18
